# Imports and preparation

In [1]:
import os

In [2]:
import numpy as np

In [3]:
import pandas as pd

In [4]:
from tqdm import tqdm

In [5]:
from transformers import AutoTokenizer #, AutoModelForSequenceClassification

In [6]:
from util import util_elastic

OpenAI tiktoken module is not available for Python < 3.8,Linux ARM64 and AARCH64. Falling back to GPT2TokenizerFast.


In [7]:
import importlib
# importlib.reload(util_elastic)

In [8]:
# Para ter repetibilidade nos resultados
random_state = 1

# Tratar valores infinitos (+ e -) como np.NaN
pd.options.mode.use_inf_as_na = True

# IMPORTANTE para tornar figuras interativas
# %matplotlib notebook

# Tamanho padrão das figuras
figsize=(10,6)

pd.set_option('display.max_row', 1000)

pd.set_option('display.max_columns', 50)

pd.set_option('display.column_space', 40)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.width', 200)


In [9]:
PATH_DOC =  '../data/juris_tcu/doc.csv'

In [10]:
%%time
import logging
logging.getLogger("haystack").setLevel(logging.WARNING) #WARNING, INFO

CPU times: user 206 µs, sys: 17 µs, total: 223 µs
Wall time: 226 µs


# Data load

In [11]:
df_doc = pd.read_csv(PATH_DOC)

In [12]:
df_doc.shape

(16045, 11)

In [13]:
df_doc.head()

,ID,TEXT,REFERENCE_LIST,PARADIGMATIC,AREA_NAME,AREA_ID_DESCRIPTOR,NORMATIVE_PROCESS_TYPE,NORMATIVE_IDENTIFICATION,NORMATIVE_DATE,NORMATIVE_AUTHOR_TYPE,NORMATIVE_AUTHOR_NAME
0,13568,"É ilegal a contagem de tempo ficto de serviço prestado sob a égide da Lei 3.313/1957, proporcion...",Lei Complementar 51/1985 || Lei Ordinária 3313/1957,NaN,Pessoal,1131,APOSENTADORIA,Acórdão 168/2014 - Segunda Câmara,2014-01-28,RELATOR,JOSÉ JORGE
1,11614,"SÚMULA TCU 283: Para fim de habilitação, a Administração Pública não deve exigir dos licitantes ...",NaN,SUMULA,Licitação,932,ADMINISTRATIVO,Acórdão 1613/2013 - Plenário,2013-06-26,RELATOR,JOSÉ JORGE
2,21087,"A contratação de serviços por preços superiores às referências legais (Sinapi e Sicro) deve ser,...",NaN,NaN,Licitação,932,PRESTAÇÃO DE CONTAS,Acórdão 3936/2013 - Segunda Câmara,2013-07-09,RELATOR,ANA ARRAES
3,35016,"Não se aplica, excepcionalmente, multa aos gestores por grave infração à norma legal se a irregu...",NaN,NaN,Responsabilidade,775,REPRESENTAÇÃO,Acórdão 3231/2011 - Plenário,2011-12-07,RELATOR,AROLDO CEDRAZ
4,29370,"Em contratatações de TI, não cabe aceitar propostas de preço com incidência de encargos majorado...",NaN,NaN,Licitação,932,REPRESENTAÇÃO,Acórdão 3231/2011 - Plenário,2011-12-07,RELATOR,AROLDO CEDRAZ


### Quantitative calculation

Tokenization loading

In [14]:
nome_modelo_ranking_pt = 'unicamp-dl/mMiniLM-L6-v2-pt-v2'
nome_caminho_modelo_pt = "/home/borela/fontes/relevar-busca/modelo/" + nome_modelo_ranking_pt
assert os.path.exists(nome_caminho_modelo_pt), f"Path para {nome_caminho_modelo_pt} não existe!"

nome_modelo_monot5_3b = 'unicamp-dl/mt5-3B-mmarco-en-pt'
# "A mono-ptT5 reranker model (850 mb) pretrained in the BrWac corpus, finetuned for 100k steps on Portuguese translated version of MS MARCO passage dataset. The portuguese dataset was translated using Google Translate.")

nome_caminho_modelo_3b = "/home/borela/fontes/relevar-busca/modelo/" + nome_modelo_monot5_3b
assert os.path.exists(nome_caminho_modelo_3b), f"Path para {nome_caminho_modelo_3b} não existe!"

tokenizador_pt_minilm = AutoTokenizer.from_pretrained(nome_caminho_modelo_pt)
tokenizador_pt_monot5_3b = AutoTokenizer.from_pretrained(nome_caminho_modelo_3b)

/home/borela/miniconda3/envs/relevar-busca/lib/python3.7/site-packages/transformers/convert_slow_tokenizer.py:447: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"


In [15]:
def retorna_num_tokens(parm_texto:str, parm_tokenizador:AutoTokenizer):
    return len(parm_tokenizador.tokenize(parm_texto))


In [16]:
df_doc['LEN_TEXT_CHAR'] = df_doc['TEXT'].apply(len)
df_doc['LEN_TEXT_CHAR_LOG'] = round(np.log(df_doc['TEXT'].apply(len))).astype(int)
df_doc['NUM_WORD'] = df_doc['TEXT'].apply(lambda x: len(x.split()))
df_doc['NUM_TOKENS_MONOT5_3B'] = df_doc['TEXT'].apply(retorna_num_tokens, parm_tokenizador=tokenizador_pt_monot5_3b)
df_doc['NUM_TOKENS_MINILM'] = df_doc['TEXT'].apply(retorna_num_tokens, parm_tokenizador=tokenizador_pt_minilm)

In [17]:
df_doc.columns

Index(['ID', 'TEXT', 'REFERENCE_LIST', 'PARADIGMATIC', 'AREA_NAME', 'AREA_ID_DESCRIPTOR', 'NORMATIVE_PROCESS_TYPE', 'NORMATIVE_IDENTIFICATION', 'NORMATIVE_DATE', 'NORMATIVE_AUTHOR_TYPE',
       'NORMATIVE_AUTHOR_NAME', 'LEN_TEXT_CHAR', 'LEN_TEXT_CHAR_LOG', 'NUM_WORD', 'NUM_TOKENS_MONOT5_3B', 'NUM_TOKENS_MINILM'],
      dtype='object')

In [18]:
# Verificar as colunas com valores NaN
columns_with_nan = df_doc.columns[df_doc.isna().any()].tolist()

print(columns_with_nan)

['REFERENCE_LIST', 'PARADIGMATIC', 'NORMATIVE_PROCESS_TYPE']


In [19]:
df_doc = df_doc.fillna("")

In [20]:
%%time
data_index_json = []
for cnt, (index, row_doc) in tqdm(enumerate(df_doc.sort_values(by='ID').iterrows()), total=df_doc.shape[0]):
    #print(row_doc['TEXT_SPECIALIZATION'], type(row_doc['TEXT_SPECIALIZATION']), np.isnan(row_doc['TEXT_SPECIALIZATION']))
    #print(isinstance(row_doc['TEXT_SPECIALIZATION'],float))
    #print(row_doc['TEXT_ENGLISH_TRANSLATION'], type(row_doc['TEXT_ENGLISH_TRANSLATION'])) #, np.isnan(row_doc['TEXT_ENGLISH_TRANSLATION']))
    try:
        row_json = {
            'content': row_doc['TEXT'],
            'id': row_doc['ID'],
            'meta': {}
        }               
        for column_name in df_doc.columns:
            if column_name not in ['TEXT']:
                row_json['meta'][column_name.lower()] = row_doc[column_name]        
        
        data_index_json.append(row_json)
    except Exception as e:
        print('row', row_doc)
        print(e)
        continue


100%|██████████| 16045/16045 [00:01<00:00, 13933.74it/s]

CPU times: user 1.15 s, sys: 11.7 ms, total: 1.16 s
Wall time: 1.16 s


In [21]:
print(f"Total {len(data_index_json)}")

Total 16045


In [22]:
print(data_index_json[0])


{'content': 'SÚMULA TCU 1: Não se compreendem como vencimento, para efeito de concessão da pensão especial com fundamento na Lei nº 3.738, de 04/04/60, as vantagens previstas no art. 184 da Lei nº 1.711, de 28/10/52.', 'id': 5, 'meta': {'id': 5, 'reference_list': 'Lei Ordinária 3738/1960 || Lei Ordinária 1711/1952, art. 184', 'paradigmatic': 'SUMULA', 'area_name': 'Pessoal', 'area_id_descriptor': 1131, 'normative_process_type': '', 'normative_identification': 'Ata 88/1973 - Plenário', 'normative_date': '1973-12-04', 'normative_author_type': 'RELATOR', 'normative_author_name': 'OCTÁVIO GALLOTTI', 'len_text_char': 203, 'len_text_char_log': 5, 'num_word': 35, 'num_tokens_monot5_3b': 63, 'num_tokens_minilm': 56}}


# Index creation

In [23]:
index_dict = util_elastic.return_indexes('indir', parm_print=True)

Index: indir_juris_tcu
{'health': 'yellow', 'status': 'open', 'index': 'indir_juris_tcu', 'uuid': 'VNWjILemS--GYDV0a4deUA', 'pri': '1', 'rep': '1', 'docs.count': '16045', 'docs.deleted': '16045', 'store.size': '334.5mb', 'pri.store.size': '334.5mb'}

Index: indir_juris_tcu_index
{'health': 'yellow', 'status': 'open', 'index': 'indir_juris_tcu_index', 'uuid': 'fGjzBrQzRUWTZ6KlnbfLqw', 'pri': '1', 'rep': '1', 'docs.count': '13255', 'docs.deleted': '2755', 'store.size': '294.9mb', 'pri.store.size': '294.9mb'}



In [24]:
INDEX_NAME = 'indir_juris_tcu'

In [26]:
# util_elastic.delete_index('indir_juris_tcu')

In [27]:
%%time
index = util_elastic.create_index(parm_index_name=INDEX_NAME,
                                  parm_data_carga_json=data_index_json,
                                  parm_embedding_dim=1024)


before write

Qtd de documentos 0

Qtd de embeddings 0

Documento.id= 5: None

after write

Qtd de documentos 16045

Qtd de embeddings 0

Documento.id= 5: <Document: id=5, content='SÚMULA TCU 1: Não se compreendem como vencimento, para efeito de concessão da pensão especial com fu...'>
CPU times: user 1.32 s, sys: 20.1 ms, total: 1.34 s
Wall time: 35.6 s


## Atualização de embeddings sem criar índice

In [28]:
%%time
index = util_elastic.return_index(parm_index_name=INDEX_NAME, parm_embedding_dim=1024)


Qtd de documentos 16045

Qtd de embeddings 0

Documento.id=5: <Document: id=5, content='SÚMULA TCU 1: Não se compreendem como vencimento, para efeito de concessão da pensão especial com fu...'>
CPU times: user 4.51 ms, sys: 0 ns, total: 4.51 ms
Wall time: 8.83 ms


In [29]:
nome_modelo_embedding_model_sts_rufimelo = "rufimelo/Legal-BERTimbau-sts-large-ma-v3"
nome_caminho_modelo = "/home/borela/fontes/relevar-busca/modelo/" + nome_modelo_embedding_model_sts_rufimelo
assert os.path.exists(nome_caminho_modelo), f"Path para {nome_caminho_modelo} não existe!"


In [30]:
%%time
util_elastic.update_index_embedding_sts(parm_index=index, parm_path_model=nome_caminho_modelo)

Updating embeddings:   0%|          | 0/16045 [00:00<?, ? Docs/s]

CPU times: user 1min 3s, sys: 2.46 s, total: 1min 5s
Wall time: 1min 58s
